In [ ]:
# Imports
from bs4 import BeautifulSoup
import requests
import csv
from urllib.parse import urljoin

In [ ]:
# Scrape lists names and links to lists from wiki (this was used for the README)
def get_lists(url):
    r = requests.get(url)
    if r.status_code != 200: raise Exception("Could not connect to '{}': code {}"
                                            .format(url, r.status_code))

    soup = BeautifulSoup(r.text, 'html.parser')

    full_table = soup.find_all('li', attrs={'class':'category-page__member'})

    return_dict = {}

    for i, list_entry in enumerate(full_table):

        for a in list_entry.find_all('a'):
            if ('Category' in a.text or    # Ran into some issues with some entries
                 a.text.isspace()):        # being empty or a link to another list of lists
                continue                   # This fixes those issues
            else:
                return_dict.update({a.text.strip('\n ') :
                                    urljoin('https://elderscrolls.fandom.com',
                                            a.get('href'))})
    return return_dict

skyrim_lists_url = 'https://elderscrolls.fandom.com/wiki/Category:Skyrim:_Lists'
dragon_born_lists_url = 'https://elderscrolls.fandom.com/wiki/Category:Dragonborn:_Lists'
hearthfire_lists_url = 'https://elderscrolls.fandom.com/wiki/Category:Hearthfire:_Lists'
creation_club_lists_url = 'https://elderscrolls.fandom.com/wiki/Category:Skyrim:_Creation_Club_Lists'

# Mash all of the lists together into one mega list
all_lists = {}
all_lists.update(get_lists(skyrim_lists_url))
all_lists.update(get_lists(dragon_born_lists_url))
all_lists.update(get_lists(hearthfire_lists_url))
all_lists.update(get_lists(creation_club_lists_url))




In [ ]:
# Generate weapon csv

# This code is a mess, wrote it very quickly. TODO complete rewrite

id_length = 8

def get_tables(url: str):
    r = requests.get(url)
    if r.status_code != 200: raise Exception("'{}' resulted in code {}".format(url, r))
    
    soup = BeautifulSoup(r.text, 'html.parser')

    tables = []

    source_tables = soup.find_all('table')
    for i, table in enumerate(source_tables):
        trs = table.find_all('tr')
        row_span = []

        results = []

        for tr in trs:
            ls = []
            for data in tr.find_all('td'):
                entry = data.text.strip('\n')
                for c in entry:
                    if ord(c) >= 128 or c == '-':
                        ls.append('N/A')
                        break
                else:
                    ls.append(entry)
                
            results.append(ls)

        for n, tr in enumerate(trs):
            tmp = []
            for i,c in enumerate(tr.find_all('td')):
                if c.has_attr('rowspan'):
                    row_span.append((n, i, int(c['rowspan']), c.text.strip('\n')))
        if row_span:
            for i in row_span:
                for j in range(1, i[2]):
                    results[i[0] + j].insert(i[1], i[3])
        tables.append(results)
    return tables

weapon_types = ['Dagger', 'Mace', 'Sword', 'War-axe', 'Other',
                'Battleaxe', 'Greatsword', 'Warhammer',
                'Bow', 'Arrow', 'Crossbow', 'Bolt']

weapons = get_tables(url='https://elderscrolls.fandom.com/wiki/Weapons_(Skyrim)')
del(weapons[-1])

for i, table in enumerate(weapons):
    
    for row in table:
        if row:
            row.insert(1, weapon_types[i])
            match i:
                case 4:
                    row.insert(5, "N/A")
                    row.insert(5, "N/A")
                    row.insert(5, "N/A")
                    row.insert(5, "N/A")
                    row.insert(5, "N/A")
                case 8:
                    row.insert(8, "N/A")
                    row.insert(8, "N/A")
                case 9:
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                    row.insert(4, 'N/A')
                case 10:
                    row.insert(5, 'N/A')
                    row.insert(7, 'N/A')
                    row.insert(8, 'N/A')
                case 11:
                    row.insert(5, 'N/A')
                    row.insert(6, 'N/A')
                    row.insert(7, 'N/A')
                    row.insert(9, 'N/A')
                case _:
                    row.insert(5, "N/A")
                    row.insert(8, "N/A")
                    row.insert(9, "N/A")
                
            if i > 7:
                row.insert(10, "Archery")
            elif i > 4:
                row.insert(10, "Two-Handed")
            else:
                row.insert(10, "One-Handed")

            row[-1] = row[-1][len(row[-1]) - id_length:]

with open('Skyrim_Weapons.csv', 'w', newline='') as weapons_csv:
    wr = csv.writer(weapons_csv, delimiter=',',
                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    wr.writerow(['name', 'category', 'damage', 'weight',
                 'value', 'speed', 'upgrade', 'perk', 
                 'description', 'additional-effects', 'type', 'id'])

    for t in weapons:
        wr.writerows(t)

